In [1]:
#Goal:
#1. Data Presentation - what is an effectve and accurate method to represent data
#2. Comparison and analysis of effective action to terminate spread of virus
#3. 3rd factor deaths of quarantine (suicide, etc.)
#4. Route and the speed of the corona virus spread
#5. timeline of news https://matplotlib.org/3.1.3/gallery/lines_bars_and_markers/timeline.html
# Number of deaths by covid (percent)

In [2]:
#comparison for number of deaths (years)

In [3]:
import sys
sys.path.append('../../myenv/lib/python3.7/site-packages')

In [4]:
from japanmap import pref_names
from japanmap import pref_code
from japanmap import groups
from japanmap import picture
import matplotlib.dates as mdates
import ipywidgets as widgets
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%config InlineBackend.figure_formats = {'png', 'retina'}
%matplotlib inline

In [5]:
path = '../../data/transformed/pcr_pop_corona_jp_v0.1.csv'
df = pd.read_csv(path, header=0)
df = df.rename(columns={"Unnamed: 0": "pref"})
df['cov_pop'] = df['Positive'] / df['population']
df = df.set_index('pref')

In [6]:
# 国内 感染確認数
Total = df['Positive'].sum()
print('陽性者数:{:,}'.format(int(Total)))
tested = df['Tested'].sum()
print('検査人数:{:,}'.format(int(tested)))

陽性者数:196,608
検査人数:3,999,511


In [7]:
def map_covid(df, col_name):
    cmap = plt.get_cmap('Reds')
    norm = plt.Normalize(vmin=df[col_name].min(), vmax=df[col_name].max())
    fcol = lambda x: '#' + bytes(cmap(norm(x), bytes=True)[:3]).hex()
    plt.colorbar(plt.cm.ScalarMappable(norm, cmap))
    plt.imshow(picture(df[col_name].apply(fcol)))
    plt.show()

In [8]:
def bar_chart(df, col_name):
    dates = df['date'].tolist()
    x_values = [datetime.datetime.strptime(d,'%Y-%m-%d').date() for d in dates]
    y_values = df[col_name].diff()

    fig, ax = plt.subplots(figsize=(10, 5))
    #auto-rotate date labels on the x-axis
    xticks = x_values[::15]
    ax.set_xticks(xticks)
    ax.set_xticklabels(xticks, rotation=60)

    ###plot values
    #format as dates
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b-%d'))
    #dates
    ax.set_xlabel('Dates: year 2020')
    ax.set_ylabel('Number of Infected People')
    ax.set_title('Active Cases')
    ax.bar(x_values, y_values)
    plt.show()

In [9]:
def tab_charts(list_graph_nms, graph_type, df):
    children = [widgets.Output() for i in range(len(list_graph_nms))]
    tab = widgets.Tab(children = children)
    for i, title in enumerate(list_graph_nms):
        tab.set_title(i, title)
        with children[i]:
            graph_type(df, title)
    
    display(tab)

In [10]:
df.columns

Index(['Positive', 'Tested', 'Percent', 'population', 'area', 'popDensity',
       'cov_pop'],
      dtype='object')

In [11]:
tab_charts(['popDensity','cov_pop', 'Percent'], map_covid, df)

In [12]:
df.sort_values(by=['popDensity'], ascending=False).head()

,Positive,Tested,Percent,population,area,popDensity,cov_pop
pref,,,,,,,
東京都,51446.0,901556.0,5.71,13971109.0,2194.07,6367.67,0.003682
大阪府,27037.0,412580.0,6.55,8817372.0,1905.32,4627.76,0.003066
神奈川県,16852.0,311376.0,5.41,9214151.0,2416.32,3813.30,0.001829
埼玉県,11675.0,281001.0,4.15,7343453.0,3797.75,1933.63,0.001590
愛知県,13967.0,170726.0,8.18,7541123.0,5173.07,1457.77,0.001852


In [13]:
path = '../../data/transformed/corona_bytime_jp_updated.csv'
df_bar = pd.read_csv(path, header=0)
df_bar = df_bar.drop(df_bar.columns[0], axis=1)
df_bar['合計'] = df_bar.sum(axis=1)
# df_bar['差'] = df_bar['合計'].diff()

In [14]:
# df_bar[(df_bar['差'] < 0)]

In [15]:
tab_charts(['東京都','静岡県','兵庫県'], bar_chart, df_bar)

In [23]:
path = '../../data/transformed/corona_bytime_jp_tested.csv'
df_tested = pd.read_csv(path, header=0)
df_tested = df_tested.drop(df_tested.columns[0], axis=1)

In [25]:
tab_charts(['東京都','静岡県','兵庫県'], bar_chart, df_tested)